In [4]:
import pandas as pd
import numpy as np

file_path = r'C:\Users\Administrator\Desktop\KissMyApps\testTasks1-2.xlsx'

# Завантаження
df = pd.read_excel(file_path)
df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])
df['revenue_usd'] = df['revenue_usd'].astype(str).str.replace(',', '.').astype(float)

In [5]:
# Додаємо тип підписки
df['period_type'] = np.where(df['product_id'].str.contains('weekly'), 'weekly',
                             np.where(df['product_id'].str.contains('monthly'), 'monthly', 'other'))

In [6]:
# Виключаємо дії після cancel
def filter_after_cancel(df):
    df = df.sort_values(by=['uuid', 'event_timestamp'])
    df['cancel_flag'] = df['event_name'] == 'cancellation'
    df['cancel_cumsum'] = df.groupby('uuid')['cancel_flag'].cumsum()
    return df[df['cancel_cumsum'] == 0]  # залишаємо лише до першого cancel

df = filter_after_cancel(df)


In [7]:
# Залишаємо тільки purchase і refund
df_filtered = df[df['event_name'].isin(['purchase', 'refund'])].copy()

# Видаляємо користувачів, у яких не було жодного purchase
has_purchase = df_filtered[df_filtered['event_name'] == 'purchase']['uuid'].unique()
df_filtered = df_filtered[df_filtered['uuid'].isin(has_purchase)]

In [8]:
# Визначаємо першу покупку як когортний івент
df_filtered['first_purchase_date'] = df_filtered[df_filtered['event_name'] == 'purchase'].groupby('uuid')['event_timestamp'].transform('min')

In [9]:
# Когорта
def calculate_ltv(df, freq='W'):
    df['cohort_period'] = df['first_purchase_date'].dt.to_period(freq)

    if freq == 'W':
        df['period_number'] = ((df['event_timestamp'] - df['first_purchase_date']) / np.timedelta64(1, 'W')).round().astype('Int64')
    elif freq == 'M':
        df['period_number'] = (
            (df['event_timestamp'].dt.year - df['first_purchase_date'].dt.year) * 12 +
            (df['event_timestamp'].dt.month - df['first_purchase_date'].dt.month)
        ).astype('Int64')
    else:
        raise ValueError("Unsupported frequency: use 'W' or 'M'")

    cohort = df.groupby(['cohort_period', 'period_number']).agg(
        revenue_usd=('revenue_usd', 'sum'),
        users=('uuid', pd.Series.nunique)
    ).reset_index()

    cohort['ltv'] = cohort['revenue_usd'] / cohort['users']
    cohort['cum_ltv'] = cohort.groupby('cohort_period')['ltv'].cumsum()

    return cohort

In [10]:
weekly_df = df_filtered[df_filtered['period_type'] == 'weekly'].copy()
monthly_df = df_filtered[df_filtered['period_type'] == 'monthly'].copy()

weekly_ltv = calculate_ltv(weekly_df, freq='W')
monthly_ltv = calculate_ltv(monthly_df, freq='M')

In [11]:
# Таблична форма
weekly_table = weekly_ltv.pivot(index='cohort_period', columns='period_number', values='cum_ltv').fillna(0).round(2)
monthly_table = monthly_ltv.pivot(index='cohort_period', columns='period_number', values='cum_ltv').fillna(0).round(2)

In [12]:
# Прогнозування на 6 періодів
def forecast_ltv(data, periods_forward=6):
    ltv_growth = data.groupby('period_number')['ltv'].mean()
    future_periods = list(range(data['period_number'].max() + 1, data['period_number'].max() + 1 + periods_forward))
    future_ltv = [ltv_growth.get(p, ltv_growth.mean()) for p in future_periods]
    return np.cumsum(future_ltv).round(2)

In [13]:
print("Таблиця Weekly LTV:")
print(weekly_table)

Таблиця Weekly LTV:
period_number            0     1      2      3      4      5      6      7   \
cohort_period                                                                 
2023-02-27/2023-03-05  3.33  6.82  10.34  13.72  17.16  20.53  24.11  27.99   
2023-03-06/2023-03-12  3.46  7.11  10.66  14.48  18.05  21.95  26.26  30.49   
2023-03-13/2023-03-19  3.46  6.89  10.54  14.13  17.81  22.61  26.47  30.24   
2023-03-20/2023-03-26  3.25  6.84  10.25  13.78  17.08  20.57  24.02  27.38   
2023-03-27/2023-04-02  3.45  7.13  10.69  14.50  18.34  22.44  26.10  29.63   
2023-04-03/2023-04-09  3.53  7.47  11.36  14.91  18.66  22.10  26.07  29.98   
2023-04-10/2023-04-16  3.49  7.40  11.03  14.61  18.25  21.60  25.34  28.80   
2023-04-17/2023-04-23  3.59  7.77  11.72  16.23  20.12  24.18  28.17  32.19   
2023-04-24/2023-04-30  3.39  7.08  10.77  14.33  18.07  21.88  25.78  29.63   
2023-05-01/2023-05-07  3.39  6.88  10.26  13.72  17.04  20.34  23.66  27.09   
2023-05-08/2023-05-14  3.66  7.7

In [14]:
print("\nПрогноз (6 тижнів):", forecast_ltv(weekly_ltv, 6))


Прогноз (6 тижнів): [ 3.62  7.24 10.85 14.47 18.09 21.71]


In [15]:
print("\nТаблиця Monthly LTV:")
print(monthly_table)


Таблиця Monthly LTV:
period_number     0      1      2       3       4       5       6       7  \
cohort_period                                                               
2023-03        8.89  31.11  68.57  103.75  139.66  173.17  203.39  238.69   
2023-04        8.89  30.77  63.65   99.40  131.64  167.95  205.97  219.04   
2023-05        8.95  30.41  62.72   97.05  131.07  157.65  173.53    0.00   
2023-06        8.96  34.67  67.67  105.67  136.47  151.98    0.00    0.00   
2023-07        9.43  31.20  54.67   94.63  104.62    0.00    0.00    0.00   
2023-08        9.16  21.64   0.00    0.00    0.00    0.00    0.00    0.00   
2023-09        8.12  31.18  39.92    0.00    0.00    0.00    0.00    0.00   
2023-10        9.99   0.00   0.00    0.00    0.00    0.00    0.00    0.00   

period_number       8  
cohort_period          
2023-03        250.37  
2023-04          0.00  
2023-05          0.00  
2023-06          0.00  
2023-07          0.00  
2023-08          0.00  
2023-09        

In [16]:
print("\nПрогноз (6 місяців):", forecast_ltv(monthly_ltv, 6))


Прогноз (6 місяців): [ 23.93  47.86  71.79  95.72 119.65 143.58]


In [ ]:
#Cеред основних рекомендацій для обрахунку LTV ще можна додатково вивести коефіцієнт відтоку 